# Spark Workflow

### Introduction

Now that we know different aspects about how and when Spark RDDs process data, let's make sure we understand the different components of what occurs when we call a Spark action.  In short, a Spark action kicks off a job, which may consist of many stages, each of which may consist of many tasks.  Let's further define Spark jobs, stages and tasks in this lesson.

### Getting Setup (On Google Colab)

* Begin by installing some pip packages and the java development kit.

In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

* Then set the java environmental variable

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

* Then connect to a SparkSession, setting the spark ui port to `4050`.

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("films").setMaster("local[2]")
sc = SparkContext.getOrCreate(conf=conf)

* Then we need to install ngrok which will allow us to place our local spark ui on the web.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')

* And finally we get a link our Spark UI

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### The Components of Calling an Action

1. Spark Jobs 

So remember that Spark transformations do not actually act on our data, whereas actions do.  This means, if we look at our Spark UI, we'll see that the number Spark jobs is equal to the number of actions that we execute.  So if we simply call `movies_rdd.take(1)`, this kicks off a spark job.

2. Spark Stages

We also saw that a single job may have multiple stages.  We can think of our stages as a logical group of steps that can be completed at once.  For us, our stages are often divided into steps that can be performed before a shuffle, and then steps that can be performed after a shuffle.

For example, let's use our spark context to create an rdd and then perform a `groupby`.

> And then create an RDD.

In [ ]:
movies = ['dark knight', 'dunkirk', 'pulp fiction', 'avatar']
movies_rdd = sc.parallelize(movies)

In [ ]:
movies_rdd.map(lambda word: word.title()). \
groupBy(lambda title: title[0]). \
map(lambda group: (group[0], len(group[1]))).collect()

[('D', 2), ('P', 1), ('A', 1)]

We can see that the first stage involved reading the file, and grouping together the data.  This is often called the `ShuffleMapStage`, because it's where our shuffling and mapping occurs.

<img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/preshuffle.png?raw=1" width="30%">

And then once our data was grouped together properly, Spark could prepare the final results of counting the number of movies by record.  This is called the `ResultStage`.

<img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/resultstage.png?raw=1" width="35%">

3. Spark Tasks

We know that each stage is performed across multiple partitions of data simultaneously.  The processing of a stage that occurs on each separate partition is called a task.  We see this when we look in the event timeline of Spark.

<img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/tasks.png?raw=1" width="100%">

Above is the tasks for a single stage, and we see that the same processing occurred across multiple partitions, with one task per partition.

### Summary

In this lesson, we saw how that spark job many have many stages and that a stage may be performed through many tasks.  There is one job per execution of an action in Spark.  And a job may contain many stages.  Oftentimes, if there is a shuffle, the stages are divided into the operations that occur before the shuffle, and the operations that occur after the shuffle -- the ShuffleMapStage and the ResultStage.  Finally, we saw that each stage can have many tasks, where we may have one task per partition of our data.  We can see these jobs, stages, and tasks through our Spark UI.

### Resources

[Apache Spark Core Youtube](https://www.youtube.com/watch?v=_ArCesElWp8&ab_channel=Databricks)